In [1]:
import torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from  torchvision import models, transforms, datasets
import os

In [2]:
data_dir = '/home/jeet/WEBEmo/'

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:
BATCH_SIZE = 64

dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
         for x in ['train', 'test']}

dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=16)
                for x in ['train', 'test']}

dset_sizes = {x: len(dsets[x]) for x in ['train', 'test']}

dset_classes = dsets['train'].classes

In [7]:
def train(model, criterion, optimizer, num_epochs = 20):
    
    for epoch in range(num_epochs):
        model.train()
        
        best_Acc = 0.0
        epoch_acc = 0.0
        epoch_loss = 0.0
        train_loss = 0.0
        train_acc = 0.0
        
        for i, (images, labels) in enumerate(dset_loaders['train']):
            
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        
            optimizer.zero_grad()
            outputs = model(images)
            
            _, preds = torch.max(outputs, 1)
            
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * images.size(0)
            train_acc += torch.sum(preds == labels.data)
            
        epoch_loss = train_loss / (BATCH_SIZE * len(dset_loaders['train']))
        epoch_acc = train_acc.double() / (BATCH_SIZE * len(dset_loaders['train']))
        
        print ("Epoch: {}, Epoch_Accuracy: {:.2f}, Epoch_loss: {:.4f}".format(epoch, epoch_acc, epoch_loss))
        
        # Evaluate on the test set
        test_acc = test()

        # Save the model if the test acc is greater than our current best
        if test_acc > best_acc:
            best_model_wt = model.state_dict()
            print("Chekcpoint updated")
            best_acc = test_acc

        # Print the metrics
        print("Test Accuracy: {}".format(test_acc))
        
    #Save the best model weights
    torch.save(best_model_wt, "Resnet34_Transfer_Learning.model")

In [8]:
def test():
    model.eval()
    test_acc = 0.0
    for i, (images, labels) in enumerate(dset_loaders['test']):

        images = Variable(images.cuda())
        labels = Variable(labels.cuda())

        # Predict classes using images from the test set
        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)
        
        test_acc += torch.sum(prediction == labels.data)

    # Compute the average acc and loss over all test images
    test_acc = test_acc.double() / (len(dset_loaders['test'])* BATCH_SIZE)

    return test_acc    

In [9]:
model_features = models.resnet34(pretrained=True)
num_features = model_features.fc.in_features
model_features.fc = nn.Linear(num_features, 25)

model_features.cuda()

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model_features.parameters(), lr=0.001, weight_decay=0.0001)

In [ ]:
train(model_features, criterion, optimizer, num_epochs=10)